In [ ]:
# !pip install -q tensorflow-gpu==2.4.0-rc1

In [1]:
import tensorflow as tf
from tensorflow.keras.layers import Dense, Flatten, Conv2D
from tensorflow.keras import Model

In [2]:
tf.__version__ 

'2.4.0'

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

# 채널 차원을 추가합니다.
x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]

In [4]:
x_train.shape #채널 차원 추가됨

(60000, 28, 28, 1)

In [5]:
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(32)
test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(32)

# 데이터셋을 섞고 배치를 만듦
# tf.data

In [6]:
class MyModel(Model):
    def __init__(self):
        super(MyModel, self).__init__()
        self.conv1 = Conv2D(32, 3, activation='relu')
        self.flatten = Flatten()
        self.d1 = Dense(128, activation='relu')
        self.d2 = Dense(10, activation='softmax')

    def call(self, x):
        x = self.conv1(x)
        x = self.flatten(x)
        x = self.d1(x)
        return self.d2(x)

model = MyModel()

In [7]:
loss_object = tf.keras.losses.SparseCategoricalCrossentropy()

optimizer = tf.keras.optimizers.Adam()


In [8]:
train_loss = tf.keras.metrics.Mean(name='train_loss')
train_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='train_accuracy')

test_loss = tf.keras.metrics.Mean(name='test_loss')
test_accuracy = tf.keras.metrics.SparseCategoricalAccuracy(name='test_accuracy')

In [9]:
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        predictions = model(images)
        loss = loss_object(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss(loss)
    train_accuracy(labels, predictions)

In [10]:
@tf.function
def test_step(images, labels):
    predictions = model(images)
    t_loss = loss_object(labels, predictions)

    test_loss(t_loss)
    test_accuracy(labels, predictions)
      

In [11]:
EPOCHS = 5

for epoch in range(EPOCHS):
    for images, labels in train_ds:
            train_step(images, labels)
    for test_images, test_labels in test_ds:
         test_step(test_images, test_labels)

    template = '에포크: {}, 손실: {}, 정확도: {}, 테스트 손실: {}, 테스트 정확도: {}'
    print (template.format(epoch+1,
                         train_loss.result(),
                         train_accuracy.result()*100,
                         test_loss.result(),
                         test_accuracy.result()*100))

에포크: 1, 손실: 0.14191579818725586, 정확도: 95.69666290283203, 테스트 손실: 0.07809522747993469, 테스트 정확도: 97.30999755859375
에포크: 2, 손실: 0.09313604235649109, 정확도: 97.17416381835938, 테스트 손실: 0.06481516361236572, 테스트 정확도: 97.79999542236328
에포크: 3, 손실: 0.06977410614490509, 정확도: 97.86166381835938, 테스트 손실: 0.06611376255750656, 테스트 정확도: 97.8133316040039
에포크: 4, 손실: 0.05603061988949776, 정확도: 98.2754135131836, 테스트 손실: 0.06489279121160507, 테스트 정확도: 97.93499755859375
에포크: 5, 손실: 0.04678637161850929, 정확도: 98.55066680908203, 테스트 손실: 0.06620559096336365, 테스트 정확도: 97.96199798583984
